In [14]:
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import SparkConf, StorageLevel
import pyspark.sql.functions as F
import os

from dotenv import load_dotenv
load_dotenv()

print(_find_spark_home())

C:\Users\anama\anaconda3\envs\Bigdata\Lib\site-packages\pyspark


In [15]:
python_path = os.environ.get('PYTHON_PATH')
app_name_dec = os.environ.get('APP_NAME_DEC')
hadoop_path_dec = os.environ.get('HADOOP_DEC_DATASET_PATH')

In [16]:
conf = SparkConf()\
    .setMaster('local[*]')\
    .set('spark-local-dir', "C:\\spark-temp")\
    .set('spark.driver.memory', '7g')\
    .set('spark.executor.memory', '7g')\
    .set('spark.driver.maxResultSize', '1g')\
    .set('spark.pyspark.python', python_path)\
    .set('spark.pyspark.driver.python', python_path)\
    .set("spark.network.timeout","800s")\
    .set("spark.dynamicAllocation.enabled", "true")\
    .set("spark.shuffle.service.enabled", "true")\
    .set("spark.dynamicAllocation.minExecutors", "1")\
    .set("spark.dynamicAllocation.maxExecutors", "10")\
    .set("spark.dynamicAllocation.executorIdleTimeout", "60s")\

spark = SparkSession.builder.appName(app_name_dec).config(conf=conf).getOrCreate()
sc = spark.sparkContext

for item in sc.getConf().getAll(): print(item)

('spark.driver.host', '194.47.46.114')
('spark.dynamicAllocation.minExecutors', '1')
('spark.app.id', 'local-1702898982487')
('spark.shuffle.service.enabled', 'true')
('spark.dynamicAllocation.maxExecutors', '10')
('spark.executor.id', 'driver')
('spark.driver.maxResultSize', '1g')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.c

In [17]:
df = spark.read.csv(hadoop_path_dec, header = True, inferSchema = True)
df.show()

In [ ]:
df_index = df.select('*').withColumn('id', F.monotonically_increasing_id())

RuntimeError: SparkContext or SparkSession should be created first.

In [ ]:
df_new = df.drop('user_session', 'product_id','category_id')
df_new.show()

+-------------------+----------+--------------------+-------+-------+---------+
|         event_time|event_type|       category_code|  brand|  price|  user_id|
+-------------------+----------+--------------------+-------+-------+---------+
|2019-12-01 01:00:00|      view|construction.tool...|  apple|1302.48|556695836|
|2019-12-01 01:00:00|      view|                null|  force| 102.96|577702456|
|2019-12-01 01:00:01|      view|appliances.person...|  bosch| 313.52|539453785|
|2019-12-01 01:00:02|  purchase|computers.periphe...|   null| 132.31|535135317|
|2019-12-01 01:00:02|      view|    apparel.trousers|   nika| 101.68|517987650|
|2019-12-01 01:00:02|      view|accessories.umbrella|   ikea| 163.56|542860793|
|2019-12-01 01:00:02|      view|  electronics.clocks|   null|  88.81|538021416|
|2019-12-01 01:00:03|      view|construction.tool...| xiaomi| 256.38|525740700|
|2019-12-01 01:00:04|      view|  computers.notebook|    jet|  20.57|512509221|
|2019-12-01 01:00:04|      view|construc

In [ ]:
df_new = df_new.dropna()
df_new.show()

+-------------------+----------+--------------------+---------+-------+---------+
|         event_time|event_type|       category_code|    brand|  price|  user_id|
+-------------------+----------+--------------------+---------+-------+---------+
|2019-12-01 01:00:00|      view|construction.tool...|    apple|1302.48|556695836|
|2019-12-01 01:00:01|      view|appliances.person...|    bosch| 313.52|539453785|
|2019-12-01 01:00:02|      view|    apparel.trousers|     nika| 101.68|517987650|
|2019-12-01 01:00:02|      view|accessories.umbrella|     ikea| 163.56|542860793|
|2019-12-01 01:00:03|      view|construction.tool...|   xiaomi| 256.38|525740700|
|2019-12-01 01:00:04|      view|  computers.notebook|      jet|  20.57|512509221|
|2019-12-01 01:00:04|      view|  computers.notebook|    vegas|  49.94|554369617|
|2019-12-01 01:00:04|      view|construction.tool...|    apple|1312.52|579969851|
|2019-12-01 01:00:06|      view|construction.tool...|  samsung| 124.11|532554953|
|2019-12-01 01:0

In [ ]:
print("Null values present in:")
for c in ["category_code", "brand"]:
    print(c +':', df_new.where(F.col(c).isNull()).count())

Null values present in:
category_code: 0
brand: 0


In [ ]:
count = df_new.count()
print(f"Total number of rows: {count:,d}")

Total number of rows: 53,612,307


In [ ]:
save_path = hadoop_path_dec + '/selected_Data/' 

df_new.repartition(8).write.mode('overwrite').csv(save_path)
spark.stop()